In [1]:
import pandas as pd
from tensorflow.keras.preprocessing import image

import tensorflow as tf

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet_v2 import ResNet152V2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import activations
from tqdm import tqdm
import numpy as np

import seaborn as sns


In [2]:
#Unagumented
df = pd.read_csv('./Unaug_Data/HAM10000_Metadata')

In [3]:
#Unaugmented
NUM_VAL = 1000
NUM_TRN = 10015 - 1000
TOTAL = 10015

In [4]:
df = df.sample(frac = 1).reset_index()

df.head()

,index,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,4805,HAM_0002101,ISIC_0029398,nv,follow_up,35.0,female,upper extremity,vidir_molemax
1,8641,HAM_0000851,ISIC_0025074,nv,histo,50.0,female,back,rosendahl
2,1274,HAM_0006803,ISIC_0032400,mel,histo,60.0,female,abdomen,vidir_modern
3,9774,HAM_0005389,ISIC_0031012,akiec,histo,70.0,male,lower extremity,rosendahl
4,3957,HAM_0005326,ISIC_0026145,nv,follow_up,45.0,female,trunk,vidir_molemax


In [5]:
#Unagumented
risk = []

for i in range(0, 10015):
    if df.iloc[i]['dx'] in ['bkl', 'df', 'vasc']:
        risk.append(0.0)
    elif df.iloc[i]['dx'] == 'nv':
        risk.append(1.0)
    elif df.iloc[i]['dx'] == 'akiec':
        risk.append(2.0)
    elif df.iloc[i]['dx'] in ['mel', 'bcc']:
        risk.append(3.0)
        
df['risk'] = risk

df.head()

,index,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,risk
0,4805,HAM_0002101,ISIC_0029398,nv,follow_up,35.0,female,upper extremity,vidir_molemax,1.0
1,8641,HAM_0000851,ISIC_0025074,nv,histo,50.0,female,back,rosendahl,1.0
2,1274,HAM_0006803,ISIC_0032400,mel,histo,60.0,female,abdomen,vidir_modern,3.0
3,9774,HAM_0005389,ISIC_0031012,akiec,histo,70.0,male,lower extremity,rosendahl,2.0
4,3957,HAM_0005326,ISIC_0026145,nv,follow_up,45.0,female,trunk,vidir_molemax,1.0


In [6]:
np.set_printoptions(threshold=np.inf)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

earlystop_callback = EarlyStopping(
  monitor='val_accuracy', min_delta=0.001,
  patience=3, restore_best_weights=True)

In [ ]:
y_trn = np.array(df['risk'])[:(-1 * NUM_VAL)]
y_val = np.array(df['risk'])[(-1 * NUM_VAL):]

# Preprocessing VGG

In [ ]:
# VGG-16
preprocessed = []

VGG_load = VGG16(weights='imagenet', include_top=False)
model = models.Sequential()
model.add(VGG_load)
model.add(layers.Flatten())


for i in tqdm(range(0, TOTAL)):
    file = df.iloc[i]['image_id']
    
    img = image.load_img('./Data/HAM10000_images/' + file + '.jpg', target_size= (224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg16.preprocess_input(img)
    preds = model.predict(img).reshape(25088)

    preprocessed.append(preds)
'''mx = 0
for i in range(0, TOTAL):
    if mx < np.amax(preprocessed[i]):
        mx = np.amax(preprocessed[i])

preprocessed = list(preprocessed / mx)
'''
VGG_X = preprocessed

# Run Training Model

In [ ]:
VGG_X_val = np.array(VGG_X)

In [ ]:
VGG_X_trn = np.array(VGG_X)[:(-1 * NUM_VAL)]

VGG_X_val = np.array(VGG_X)[(-1 * NUM_VAL):]

del VGG_X

In [ ]:
new_y_trn = []
for ele in y_trn:
    new_y_trn.append(np.array(tf.one_hot(ele, 4)))
    
new_y_trn = np.array(new_y_trn)

new_y_trn.shape

In [ ]:
new_y_val = []
for ele in y_val:
    new_y_val.append(np.array(tf.one_hot(ele, 4)))
    
new_y_val = np.array(new_y_val)

new_y_val.shape

In [ ]:
new_VGG_trn = []

for i in tqdm(range(0, NUM_TRN)):
    new_VGG_trn.append(np.array(VGG_X_trn[i]))

new_VGG_trn = np.array(new_VGG_trn)

In [ ]:
new_VGG_val = []

for i in tqdm(range(0, NUM_VAL)):
    new_VGG_val.append(np.array(VGG_X_val[i]))

new_VGG_val = np.array(new_VGG_val)

In [ ]:
vgg_model = models.Sequential()
vgg_model.add(layers.Dense(8192, activation='relu'))
vgg_model.add(layers.Dense(4096, activation='relu'))
vgg_model.add(layers.Dense(2048, activation='relu'))
vgg_model.add(layers.Dense(1024, activation='relu'))
vgg_model.add(layers.Dense(512, activation='relu'))
vgg_model.add(layers.Dense(128, activation='relu'))
vgg_model.add(layers.Dense(4))

In [ ]:
vgg_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
              loss=tf.nn.softmax_cross_entropy_with_logits,
              metrics=['accuracy'])
vgg_model.fit(new_VGG_trn, new_y_trn, 
              epochs=30, 
              validation_data=(new_VGG_val, new_y_val), 
              callbacks=[earlystop_callback])

In [ ]:
predict_y_vgg_val = []
temp_vgg_trn = vgg_model.predict(new_VGG_trn)
temp_vgg_val = vgg_model.predict(new_VGG_val)

del new_VGG_trn
del new_VGG_val

for i in tqdm(range(0, NUM_VAL)):
    pred = np.argmax(temp_vgg_val[i])
    predict_y_vgg_val.append(pred)

# Draw Heatmap

In [ ]:
cm_vgg = tf.math.confusion_matrix(np.array(df['risk'])[(-1 * NUM_VAL):], np.array(predict_y_vgg_val))

cm_vgg = np.array(cm_vgg).astype('float32')

cm_vgg[0] = cm_vgg[0] / (1.0 * cm_vgg[0].sum())
cm_vgg[1] = cm_vgg[1] / (1.0 * cm_vgg[1].sum())
cm_vgg[2] = cm_vgg[2] / (1.0 * cm_vgg[2].sum())
cm_vgg[3] = cm_vgg[3] / (1.0 * cm_vgg[3].sum())

In [ ]:
svm = sns.heatmap(cm_vgg, annot=True);

In [ ]:
figure = svm.get_figure()
figure.savefig(r"A:\Deep Learning\Term Project\Pic\Ensemble-Resnet-no adjustment\final.png", dpi = 400)